In [1]:
import uuid
import random
import pandas as pd
import numpy as np
from datetime import datetime
from create_sap_table import create_table_leanx as ctl

import values, helpers
import master_data, text_data, purchasing_doc_data

In [2]:
# # text tables
# for method in (
#     text_data.domain_fixed_values, 
#     text_data.sales_doc_types, 
#     text_data.sales_organizations,
#     text_data.distribution,
#     text_data.sales_doc_item_categories,
#     text_data.sales_doc_rejection_reasons,
#     text_data.system_status,
#     text_data.blocking_reasons,
#     text_data.releases,
#     text_data.purchasing_doc_types,
#     text_data.purchasing_organizations
# ):
#     table_dict = method()
#     for k, v in table_dict.items():
#         table_name = k.split('_')[0]
#         all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
#         df = pd.concat([all_cols, pd.DataFrame(v.values())])
#         df.to_csv(f'data/P2P/OCPM/text/{table_name}.csv', index=False)

In [3]:
# # master tables
# for method in (
#     master_data.users,
#     master_data.customers_and_vendors, 
#     master_data.plants, 
#     master_data.materials, 
#     master_data.material_support,
#     master_data.routes,
#     master_data.company_codes
# ):
#     table_dict = method()
#     for k, v in table_dict.items():
#         table_name = k.split('_')[0]
#         all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
#         df = pd.concat([all_cols, pd.DataFrame(v.values())])
#         df.to_csv(f'data/P2P/OCPM/master/{table_name}.csv', index=False)

In [4]:
MARC = pd.read_csv('data/P2P/OCPM/master/MARC.csv')
MARA = pd.read_csv('data/P2P/OCPM/master/MARA.csv')
MAKT = pd.read_csv('data/P2P/OCPM/master/MAKT.csv')
LFB1 = pd.read_csv('data/P2P/OCPM/master/LFB1.csv')
LFA1 = pd.read_csv('data/P2P/OCPM/master/LFA1.csv')

all_prices = {}
# all_availabilities = {}
# all_material_groups = {}

# get price by MATNR
for nr in MARA['MATNR']:
    name = MAKT[MAKT['MATNR'] == nr]['MAKTX'].values[0]
    for k, v in values.om_material_groups.items():
        for mgrp, attr in v.items():
            for mat, details in attr['materials'].items():
                if name == mat:
                    all_prices[nr] = details['price']
                    # all_availabilities[nr] = attr['availability']

def get_user_name(automation_probability: float):
    if random.random() <= automation_probability:
        return 'BATCH_JOB'
    return random.choice(list(values.om_users.keys()))

def get_time_consumption(start_date, planned_target_date, latest_date):
    return (latest_date - start_date) / (planned_target_date - start_date)

def get_params():
    company_code = random.choice(list(values.om_company_codes.keys()))
    plant = random.choice(values.om_company_codes[company_code]['plants'])
    all_matnrs = MARC[MARC['WERKS'] == plant]['MATNR'].unique()
    matnrs = random.sample(list(all_matnrs), min(random.randint(5, 25), len(all_matnrs)))
    quantities = [random.randint(12, 60)*12 for _ in range(len(matnrs))] # x dozens of everything
    prices = []
    # availabilities = []
    
    for i in range(len(matnrs)):
        prices.append(all_prices[matnrs[i]])
        # availabilities.append(all_availabilities[matnrs[i]])

    pr_prices = [p*(1+min(0.5,random.random())) for p in prices]
    
    konnr = f'{str(uuid.uuid4())[-15:]}{i}'
    requested_by = get_user_name(0.3)
    purchasing_org=random.choice(list(values.proc_purchasing_orgs.keys()))
    # sales_office=random.choice(list(values.om_sales_orgs[sales_org]['sales_offices'].keys()))
    # distribution_channel=random.choice(list(values.om_sales_orgs[sales_org]['distribution_channels'].keys()))
    lifnr = random.choice(list(LFB1[LFB1['BUKRS'] == company_code]['LIFNR']))
    # vendor_name = LFA1[LFA1['LIFNR'] == lifnr]['NAME1'].values[0]
    payment_term = random.choice(list(LFB1[LFB1['LIFNR'] == lifnr]['ZTERM']))
    # credit_risk = values.om_customers[customer_name]['credit_risk']
    # sales_doc_type=random.choice(list(values.om_sales_doc_types.keys()))

    # total = np.sum([prices[i]*quantities[i] for i in range(len(prices))])
    has_contract = True if random.random() < 0.5 else False
    is_free_text = True if random.random() < 0.5 else False
    has_pr_price_mismatch = True if random.random() < 0.5 else False
    free_text_materials = matnrs # TODO change with material description


    params = {
        'matnrs': matnrs,
        'konnr': konnr,
        'lifnr': lifnr,
        'plant': plant,
        'quantities': quantities,
        'prices': prices, # considered contract and PO prices
        'company_code': company_code,
        'purchasing_org': purchasing_org,
        'payment_term': payment_term,
        'requested_by': requested_by,
        'has_contract': has_contract,
        'has_pr_price_mismatch': has_pr_price_mismatch, # used for PR on price mismatch only
        'pr_prices': pr_prices,
        'is_free_text': is_free_text,
        'free_text_materials': free_text_materials,
    }

    return params

In [5]:
purchasing_doc_tables = {
    'EBAN_json': {},
    'CDHDR_json': {},
    'CDPOS_json': {},
    'EKKO_json': {},
    'EKPO_json': {},
    'NAST_json': {},
    'MSEG_json': {},
    'EKBE_json': {},
    'RBKP_json': {},
    'RSEG_json': {},
    'EKET_json': {},
    'EKES_json': {},
}
for i in range(1_000):
    params = get_params()
    latest_date = helpers.generate_random_date(start_date=datetime(2021, 1, 1), end_date=datetime(2023, 9, 1))
    latest_time = helpers.generate_random_time()
    purchasing = purchasing_doc_data.Purchasing(params=params, start_date=latest_date, index=i)
    preq_creation_date = latest_date

    purchasing.create_contract( # TODO check if 1:1 mapping between contracts, PRs, and POs creates an issue
         aedat=latest_date,
         ernam=get_user_name(0.5)
    )
    
    purchasing.create_purchase_requisition_item(
        badat=preq_creation_date, 
        ernam=get_user_name(0.5),
    )
    latest_date += helpers.UPTO_WEEK()

    purchasing.create_purchase_order(
         aedat=latest_date,
         ernam=get_user_name(0.5)
    )
    latest_date += helpers.UPTO_WEEK()

    purchasing.send_purchase_order(
         usnam=get_user_name(0.5),
         erdat=latest_date
    )
    latest_date += helpers.UPTO_WEEK()

    purchasing.post_goods_receipt(
         cpudt=latest_date,
         usnam=get_user_name(0.5),
         atime=latest_time
    )

    purchasing.approve_purchase_order(
          aedat=latest_date,
          ernam = get_user_name(0.5)
     )
    latest_date += helpers.UPTO_WEEK()

    purchasing.create_purchase_order_schedule_line(
         eindt=latest_date,
         ernam=get_user_name(0.5)
    )

    for k, v in purchasing.tables.items():
            for entry_key in list(v.keys()):
                purchasing_doc_tables[k][entry_key] = purchasing.tables[k][entry_key]

In [6]:
for table, rows in purchasing_doc_tables.items():
    table_name = table.split('_')[0]
    all_cols = pd.DataFrame(columns=[c[0] for c in ctl.fetch_table(table_name)])
    df = pd.concat([all_cols, pd.DataFrame(rows.values())])
    df.to_csv(f'data/P2P/OCPM/purchasing-document/{table_name}.csv', index=False)

In [7]:

    
#     # Vendor Confirmation based on PO Item
#     for i in range(len(params['matnrs'])): 
#         purchasing.set_confirmed_poitem_delivery_date(
#              ernam=get_user_name(0.5),
#             ebelp = i,
#             confdate = latest_date)
#         purchasing.restore_purchase_order_item(
#             badat=latest_date,
#             ernam = get_user_name(0.5),
#             ebelp = i
#             )
#         purchasing.block_purchase_order_item(
#                 badat=latest_date,
#                 ernam = get_user_name(0.5),
#                 ebelp = i
#                 )

#     purchasing.create_vendor_invoice(
#          bldat = latest_date,
#          ernam=get_user_name(0.5),
#          cputm = helpers.generate_random_time(),
#     )